In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
class Sampling(tf.keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [180]:
class VAE(tf.keras.Model):
        def __init__(self, encoder, decoder, **kwargs):
            super(VAE, self).__init__(**kwargs)
            self.encoder = encoder
            self.decoder = decoder
            self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
            self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
                name="reconstruction_loss"
            )
            self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

        @property
        def metrics(self):
            return [
                self.total_loss_tracker,
                self.reconstruction_loss_tracker,
                self.kl_loss_tracker,
            ]

        def train_step(self, data):
            with tf.GradientTape() as tape:
                z_mean, z_log_var, z = self.encoder(data)
                reconstruction = self.decoder(z)
                reconstruction_loss = tf.keras.losses.MeanSquaredError(data, reconstruction)
                kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
                kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
                total_loss = reconstruction_loss + kl_loss
            grads = tape.gradient(total_loss, self.trainable_weights)
            self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
            self.total_loss_tracker.update_state(total_loss)
            self.reconstruction_loss_tracker.update_state(reconstruction_loss)
            self.kl_loss_tracker.update_state(kl_loss)
            return {
                "loss": self.total_loss_tracker.result(),
                "reconstruction_loss": self.reconstruction_loss_tracker.result(),
                "kl_loss": self.kl_loss_tracker.result(),
            }

SyntaxError: unmatched ')' (1914429242.py, line 27)

In [181]:
class VAED2:
    
    class Sampling(tf.keras.layers.Layer):
        """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

        def call(self, inputs):
            z_mean, z_log_var = inputs
            batch = tf.shape(z_mean)[0]
            dim = tf.shape(z_mean)[1]
            epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
            return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
    
    def __init__(self,neurons_in_layers,lower_dim,input_shape=(2830),acv='relu',reg='l2'):
            
        self.neurons_in_layers = neurons_in_layers 
        self.input_shape = input_shape
        self.lower_dim = lower_dim
        self.acv = acv
        self.reg = reg
        return

    def Encoder(self):
        "layers_filter should be a list: [32,64,64] "
        inputs = tf.keras.layers.Input(shape=self.input_shape,name='Input')
        x = inputs
        for neurons in self.neurons_in_layers:
            x = tf.keras.layers.Dense(units=neurons,activation=self.acv,kernel_regularizer=self.reg)(x)
        z_mean = tf.keras.layers.Dense(self.lower_dim, name="z_mean")(x)
        z_log_var = tf.keras.layers.Dense(self.lower_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        encoder = tf.keras.Model(inputs, [z_mean, z_log_var, z], name="encoder")

        return encoder


    def Decoder(self,enc_shape):
        "layers_filter should be a list: [32,64,64] given as same to encoder "
        self.neurons_in_layers.reverse();
        lower_inputs = tf.keras.layers.Input(shape=self.lower_dim,name='Decoder_Input')
        x = tf.keras.layers.Dense(enc_shape[0],activation=self.acv,kernel_regularizer=self.reg)(lower_inputs)
        for i,neurons in enumerate(self.neurons_in_layers[:]):
            x = tf.keras.layers.Dense(units=neurons,activation=self.acv,kernel_regularizer=self.reg)(x)
            
        outputs = x

        decoder = tf.keras.Model(inputs=lower_inputs,outputs=outputs, name='Decoder')

        return decoder   


In [182]:
vae = VAED2([10,30],2,(10),'selu')

In [183]:
encoder = vae.Encoder();

In [184]:
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 10)]         0           []                               
                                                                                                  
 dense_101 (Dense)              (None, 10)           110         ['Input[0][0]']                  
                                                                                                  
 dense_102 (Dense)              (None, 30)           330         ['dense_101[0][0]']              
                                                                                                  
 z_mean (Dense)                 (None, 2)            62          ['dense_102[0][0]']              
                                                                                            

In [185]:
decoder = vae.Decoder(encoder.layers[-2].output_shape[1:])
decoder.summary()

Model: "Decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Decoder_Input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense_103 (Dense)           (None, 2)                 6         
                                                                 
 dense_104 (Dense)           (None, 30)                90        
                                                                 
 dense_105 (Dense)           (None, 10)                310       
                                                                 
Total params: 406
Trainable params: 406
Non-trainable params: 0
_________________________________________________________________


In [186]:
model = VAE(encoder,decoder)

In [187]:
# data = np.load('ar1.npy')

In [188]:
data = []
for j in range(1000):
    x = []
    for i in range(10):
        x.append(i*2)
    data.append(x)

data = np.array(data)

In [189]:
data.shape

(1000, 10)

In [190]:
# data[:,-1]

In [191]:
model.compile(optimizer='adam')

In [192]:
model.fit(data,epochs=100)

Epoch 1/100
32/32 [==============================] - 1s 2ms/step - loss: 3075.1403 - reconstruction_loss: 2498.3491 - kl_loss: 72.0280
Epoch 2/100
32/32 [==============================] - 0s 1ms/step - loss: 1438.5555 - reconstruction_loss: 1090.4528 - kl_loss: 149.0403
Epoch 3/100
32/32 [==============================] - 0s 2ms/step - loss: 749.2606 - reconstruction_loss: 481.5361 - kl_loss: 128.3701
Epoch 4/100
32/32 [==============================] - 0s 2ms/step - loss: 319.7309 - reconstruction_loss: 142.8595 - kl_loss: 133.4518
Epoch 5/100
32/32 [==============================] - 0s 2ms/step - loss: 191.0244 - reconstruction_loss: 50.4688 - kl_loss: 117.3307
Epoch 6/100
32/32 [==============================] - 0s 1ms/step - loss: 114.0759 - reconstruction_loss: 9.5685 - kl_loss: 98.2488
Epoch 7/100
32/32 [==============================] - 0s 1ms/step - loss: 91.2823 - reconstruction_loss: 5.7941 - kl_loss: 81.5597
Epoch 8/100
32/32 [==============================] - 0s 1ms/step - 